In [1]:
import requests
import json

#test用のデータ
text ="既報通り、グーグル（以下、Google Japan）は5日、都内にて「新ハードウェア製品記者発表会」を開催し、Googleアシスタントを搭載したスマートディスプレイ「Google Nest Hub」を6月12日に税抜14,000円（税込15,120円）で発売すると発表した。"

dic = {"text":text}
text = json.dumps(dic)

# user_idとアクセストークンを設定
user_id = "XXXXX"
access_token = "XXXXX"
mecab_url = "XXXXX"
auth = (user_id, access_token)
response = requests.post(mecab_url, text,headers={'Content-Type': 'application/json; charset=UTF-8'}, auth=auth)

In [2]:
response.json()

'既報\t名詞,一般,*,*,*,*,既報,キホウ,キホー\n通り\t名詞,接尾,一般,*,*,*,通り,ドオリ,ドーリ\n、\t記号,読点,*,*,*,*,、,、,、\nグーグル\t名詞,固有名詞,一般,*,*,*,グーグル,グーグル,グーグル\n（\t記号,括弧開,*,*,*,*,（,（,（\n以下\t名詞,非自立,副詞可能,*,*,*,以下,イカ,イカ\n、\t記号,読点,*,*,*,*,、,、,、\nGoogle\t名詞,固有名詞,一般,*,*,*,Google,グーグル,グーグル\nJapan\t名詞,固有名詞,地域,一般,*,*,Japan,ジャパン,ジャパン\n）\t記号,括弧閉,*,*,*,*,）,）,）\nは\t助詞,係助詞,*,*,*,*,は,ハ,ワ\n5日\t名詞,固有名詞,一般,*,*,*,5日,イツカ,イツカ\n、\t記号,読点,*,*,*,*,、,、,、\n都内\t名詞,一般,*,*,*,*,都内,トナイ,トナイ\nにて\t助詞,格助詞,一般,*,*,*,にて,ニテ,ニテ\n「\t記号,括弧開,*,*,*,*,「,「,「\n新\t接頭詞,名詞接続,*,*,*,*,新,シン,シン\nハードウェア\t名詞,一般,*,*,*,*,ハードウェア,ハードウェア,ハードウェア\n製品\t名詞,一般,*,*,*,*,製品,セイヒン,セイヒン\n記者発表\t名詞,固有名詞,一般,*,*,*,記者発表,キシャハッピョウ,キシャハッピョー\n会\t名詞,接尾,一般,*,*,*,会,カイ,カイ\n」\t記号,括弧閉,*,*,*,*,」,」,」\nを\t助詞,格助詞,一般,*,*,*,を,ヲ,ヲ\n開催\t名詞,サ変接続,*,*,*,*,開催,カイサイ,カイサイ\nし\t助詞,接続助詞,*,*,*,*,し,シ,シ\n、\t記号,読点,*,*,*,*,、,、,、\nGoogleアシスタント\t名詞,固有名詞,一般,*,*,*,Googleアシスタント,グーグルアシスタント,グーグルアシスタント\nを\t助詞,格助詞,一般,*,*,*,を,ヲ,ヲ\n搭載\t名詞,サ変接続,*,*,*,*,搭載,トウサイ,トーサイ\nした\t名詞,一般,*,*,*,*,した,シタ,シタ\nスマートディスプレイ\t名詞,固有名詞,一般,*,*,*,スマートディスプレイ,スマートディスプ

In [3]:
#前処理、名詞のみ対象として解析を実施する
word_list = []
doc_list = []

text_data = response.json()
text_data_list = text_data.split("\n")
    
for text in text_data_list:
    if text == 'EOS':
        break
    else:
        text = text.split("\t")
        word = text[0]
        word_meta = text[1]
                
        word_detail = word_meta.split(",")
        if word_detail[0] == '名詞':
            doc_list.append(word)
doc_list = [doc_list]

In [4]:
#gensimのinstall
!pip install gensim

Looking in indexes: https://pypi.org/simple, https://packagecloud.io/abeja/platform-public/pypi/simple
You are using pip version 19.0.3, however version 19.1.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [6]:
# 辞書のLOAD
from gensim.corpora import Dictionary
dic = Dictionary.load_from_text("../model/livedoordic.txt")
bow_corpus = [dic.doc2bow(d) for d in doc_list]

In [7]:
# tfidfモデルのLOAD
from gensim.models import TfidfModel
tfidf_model  = TfidfModel.load("../model/tfidf_model.model")
tfidf_corpus = tfidf_model[bow_corpus]

In [8]:
# LSIモデルのロード
from gensim.models import LsiModel
from gensim import models
lsi_model = LsiModel.load("../model/lsi_model.model")
lsi_corpus = lsi_model[tfidf_corpus]

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [14]:
# ラベルの定義
label = ['dokujo-tsushin',
 'it-life-hack',
 'kaden-channel',
 'livedoor-homme',
 'movie-enter',
 'peachy',
 'smax',
 'sports-watch',
 'topic-news']

In [15]:
#gensimコーパスからdenseへ
from gensim import matutils
dime = 100
test_dense = list(matutils.corpus2dense(lsi_corpus, num_terms=dime).T)

In [17]:
#SVCモデルのロードと推論
import pickle
filename = '../model/SVC_model.sav'
loaded_model = pickle.load(open(filename, 'rb'))
result = loaded_model.predict(test_dense)
print(label[int(result)])

peachy


In [18]:
#XGBoostモデルのロードと推論
filename = '../model/Xgboost_model.sav'
loaded_model = pickle.load(open(filename, 'rb'))
result = loaded_model.predict(test_dense)
print(label[int(result)])

it-life-hack


In [19]:
#LightGBMモデルのロードと推論
import numpy as np
filename = '../model/LightGBM_model.sav'
loaded_model = pickle.load(open(filename, 'rb'))
result_list = loaded_model.predict(test_dense)
result = np.argmax(result_list, axis=1)
print(label[int(result)])

topic-news
